In [ ]:
#/FileStore/tables/hist_data_csv-72c06.txt
#/FileStore/tables/incr_data.csv
from pyspark.sql.types import *
hist_df=spark.read.format("csv").option("header", "true").load("/FileStore/tables/hist_data_csv-72c06.txt")
incr_df=spark.read.format("csv").option("header", "true").load("/FileStore/tables/incr_data.csv")

#hist_df.show()
#incr_df.show()
#hist_df.printSchema()
#incr_df.printSchema()

In [2]:
from pyspark.sql.functions import col
from pyspark.sql.functions import *
df_inter=incr_df.alias("t1").join(hist_df.alias("t2"),col("t1.Id") == col("t2.Id"),'inner').select(col("t1.Id"),col("t1.Name"),col("t1.Place"),col("t1.Eff_start"),col("t2.Eff_end"),col("t2.Indicator"))
df_inter.show()

df_new=incr_df.alias("t1").join(hist_df.alias("t2"),col("t1.Id") ==col("t2.Id"),'left').filter(col("t2.Id").isNull()).select(col("t1.Id"),col("t1.Name"),col("t1.Place"),col("t1.Eff_start"),col("t2.Eff_end"),col("t2.Indicator")).withColumn("Eff_end",lit("9999-12-31")).withColumn("Indicator",lit("1"))

df_new.show()

df_old = hist_df.alias("t1").join(incr_df.alias("t2"),col("t1.Id") == col("t2.Id"),'left').select("t1.*").filter(col("t2.Id").isNull())
df_old.show()

df_old_exp = incr_df.alias("t1").join(hist_df.alias("t2"),col("t1.Id") == col("t2.Id"),'inner').select("t2.*").withColumn("Eff_end",current_date()).withColumn("Indicator",lit("0"))
df_old_exp.show()


df_final=df_new.unionAll(df_inter).unionAll(df_old_exp).unionAll(df_old)
df_final.orderBy("Id").show()


+---+----+--------+----------+----------+---------+
 Id|Name| Place| Eff_start| Eff_end|Indicator|
+---+----+--------+----------+----------+---------+
 2|Hari| Hyd|2019-11-15|9999-12-31| 1|
 3|Nick|Banglore|2019-11-15|9999-12-31| 1|
+---+----+--------+----------+----------+---------+

+---+----+-----+----------+----------+---------+
 Id|Name|Place| Eff_start| Eff_end|Indicator|
+---+----+-----+----------+----------+---------+
 6|John| Hyd|2019-11-16|9999-12-31| 1|
+---+----+-----+----------+----------+---------+

+---+----+--------+----------+----------+---------+
 Id|Name| Place| Eff_start| Eff_end|Indicator|
+---+----+--------+----------+----------+---------+
 1|Liji|Banglore|2019-10-10|9999-12-31| 1|
 4| Ram| Chennai|2019-10-13|9999-12-31| 1|
 5| Sam| Hyd|2019-10-14|9999-12-31| 1|
+---+----+--------+----------+----------+---------+

+---+----+--------+----------+----------+---------+
 Id|Name| Place| Eff_start| Eff_end|Indicator|
+---+----+--------+----------+----------+---------+
 2|Hari| Chennai|2019-10-11|2020-04-24| 0|
 3|Nick|Banglore|2019-10-12|2020-04-24| 0|
+---+----+--------+----------+----------+---------+

+---+----+--------+----------+----------+---------+
 Id|Name| Place| Eff_start| Eff_end|Indicator|
+---+----+--------+----------+----------+---------+
 1|Liji|Banglore|2019-10-10|9999-12-31| 1|
 2|Hari| Chennai|2019-10-11|2020-04-24| 0|
 2|Hari| Hyd|2019-11-15|9999-12-31| 1|
 3|Nick|Banglore|2019-11-15|9999-12-31| 1|
 3|Nick|Banglore|2019-10-12|2020-04-24| 0|
 4| Ram| Chennai|2019-10-13|9999-12-31| 1|
 5| Sam| Hyd|2019-10-14|9999-12-31| 1|
 6|John| Hyd|2019-11-16|9999-12-31| 1|
+---+----+--------+----------+----------+---------+

In [4]:
from pyspark.sql.functions import col
from pyspark.sql.functions import *
df_newrec = incr_df.alias("t1").join(hist_df.alias("t2"),col("t1.Id") == col("t2.Id"),"left").filter(col("t2.Id").isNull()).select("t1.*",col("t2.Eff_end"),col("t2.Indicator")).withColumn("Eff_end",lit("9999-12-31")).withColumn("Indicator",lit("1"))
#.withCo)lumn("Eff_end","9999-12-31").withColumn("Indicator","1")

df_oldrec = incr_df.alias("t1").join(hist_df.alias("t2"),col("t1.Id") == col("t2.Id"),"right").filter(col("t1.Id").isNull()).select("t2.*")
df_uprec=incr_df.alias("t1").join(hist_df.alias("t2"),col("t1.Id")==col("t2.Id"),"inner").select("t1.*",col("t2.Eff_end"),col("t2.Indicator")).withColumn("Eff_End",lit("9999-12-31")).withColumn("Indicator",lit("1"))
df_uprec.show()
#.withColumn("Eff_end",lit("9999-12-31")).withColumn("Indicator",lit("1"))
df_oldrec.show()
df_newrec.show()

dffinal=df_newrec.unionAll(df_oldrec).unionAll(df_uprec)
dffinal.show()

+---+----+--------+----------+----------+---------+
 Id|Name| Place| Eff_start| Eff_End|Indicator|
+---+----+--------+----------+----------+---------+
 2|Hari| Hyd|2019-11-15|9999-12-31| 1|
 3|Nick|Banglore|2019-11-15|9999-12-31| 1|
+---+----+--------+----------+----------+---------+

+---+----+--------+----------+----------+---------+
 Id|Name| Place| Eff_start| Eff_end|Indicator|
+---+----+--------+----------+----------+---------+
 1|Liji|Banglore|2019-10-10|9999-12-31| 1|
 4| Ram| Chennai|2019-10-13|9999-12-31| 1|
 5| Sam| Hyd|2019-10-14|9999-12-31| 1|
+---+----+--------+----------+----------+---------+

+---+----+-----+----------+----------+---------+
 Id|Name|Place| Eff_start| Eff_end|Indicator|
+---+----+-----+----------+----------+---------+
 6|John| Hyd|2019-11-16|9999-12-31| 1|
+---+----+-----+----------+----------+---------+

+---+----+--------+----------+----------+---------+
 Id|Name| Place| Eff_start| Eff_end|Indicator|
+---+----+--------+----------+----------+---------+
 6|John| Hyd|2019-11-16|9999-12-31| 1|
 1|Liji|Banglore|2019-10-10|9999-12-31| 1|
 4| Ram| Chennai|2019-10-13|9999-12-31| 1|
 5| Sam| Hyd|2019-10-14|9999-12-31| 1|
 2|Hari| Hyd|2019-11-15|9999-12-31| 1|
 3|Nick|Banglore|2019-11-15|9999-12-31| 1|
+---+----+--------+----------+----------+---------+